In [2]:
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, pacf
from datetime import timedelta
from sklearn.metrics import root_mean_squared_error
import pickle


In [3]:
with open('api_temp_sarima.pkl', 'rb') as file:
    api_temp_model = pickle.load(file)

with open('humidity_sarima.pkl', 'rb') as file:
    humidity_model = pickle.load(file)

with open('temperature_sarima.pkl', 'rb') as file:
    temperature_model = pickle.load(file)

In [4]:
with open('plant_water.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

In [5]:
loaded_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:          soil_moisture   No. Observations:                  146
Model:               SARIMAX(2, 0, 0)   Log Likelihood                -622.814
Date:                Sat, 19 Apr 2025   AIC                           1257.627
Time:                        21:47:41   BIC                           1275.529
Sample:                             0   HQIC                          1264.901
                                - 146                                         
Covariance Type:                  opg                                         
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
temperature    25.4523      4.079      6.240      0.000      17.457      33.447
humidity       10.1663      0.402     25.298      0.000       9.379      10.954
api_temp       52.1151      3.594     14.499      0.000      45.070      59.160
ar.L1           0.5952      0.090      6.619      0.000       0.419       0.771
ar.L2           0.2124      0.089      2.385      0.017       0.038       0.387
sigma2        295.1162     30.308      9.737      0.000     235.713     354.519
===================================================================================
Ljung-Box (L1) (Q):                   0.03   Jarque-Bera (JB):                 9.94
Prob(Q):                              0.87   Prob(JB):                         0.01
Heteroskedasticity (H):               1.11   Skew:                             0.40
Prob(H) (two-sided):                  0.71   Kurtosis:                         3.99
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [6]:
df = pd.read_csv("dataset.csv", index_col="ts", parse_dates = True)
df.head()

,id,light,temperature,soil_moisture,humidity,api_temp,precipitation,cloud_cover
ts,,,,,,,,
2025-04-01 21:15:00,1,34.3262,33.3125,3262.0,84,27.9,0.1,100
2025-04-01 21:30:00,2,33.9057,33.4688,3232.5,84,27.8,0.1,100
2025-04-01 21:45:00,3,33.2979,33.5938,3218.0,85,27.8,0.1,100
2025-04-01 22:00:00,4,33.6201,33.3281,3211.0,85,27.6,0.0,100
2025-04-01 22:15:00,5,33.2979,33.3438,3203.0,85,27.6,0.0,100


In [23]:
help(api_temp_model.get_prediction)

Help on method get_prediction in module statsmodels.tsa.statespace.mlemodel:

get_prediction(start=None, end=None, dynamic=False, information_set='predicted', signal_only=False, index=None, exog=None, extend_model=None, extend_kwargs=None, **kwargs) method of statsmodels.tsa.statespace.sarimax.SARIMAXResults instance
    In-sample prediction and out-of-sample forecasting

    Parameters
    ----------
    start : int, str, or datetime, optional
        Zero-indexed observation number at which to start forecasting,
        i.e., the first forecast is start. Can also be a date string to
        parse or a datetime type. Default is the the zeroth observation.
    end : int, str, or datetime, optional
        Zero-indexed observation number at which to end forecasting, i.e.,
        the last forecast is end. Can also be a date string to
        parse or a datetime type. However, if the dates index does not
        have a fixed frequency, end must be an integer index if you
        want out

In [20]:
forecast_steps = 200
last_date = df.index[-1]
last_date + timedelta(2)

Timestamp('2025-04-16 11:15:00')

In [25]:
# After converting index to DatetimeIndex
forecast_steps = 200
last_date = df.index[-1]

# Generate proper date range (adjust freq='T' for 5-minute intervals etc.)
dates = pd.date_range(
    start=last_date,
    periods=forecast_steps+1,  # Includes start point
    freq='D'                   # Change to your actual frequency (e.g., '5T')
)[1:]  # Exclude first point (last known date)

predicted_values = api_temp_model.get_prediction(
    start=last_date,
    end=last_date + datetime(2)
)
prediction = pd.DataFrame(predicted_values, columns=['predicted'])

print(prediction)


TypeError: function missing required argument 'month' (pos 2)

In [20]:
# Define start and end for forecast (e.g., next 200 steps after observed data)
start = int(df['soil_moisture'].count())
end = start + 200

# Select exogenous variables for the forecast period
# Make sure this slice corresponds exactly to the forecast horizon
exog_forecast = df[['temperature', 'humidity', 'api_temp']].iloc[start:end+1]

# Call predict with exog inside the method
predicted_values = loaded_model.predict(start=start, end=end, exog=exog_forecast)

# Convert to DataFrame and rename column
prediction = pd.DataFrame(predicted_values, columns=['predicted'])

print(prediction)


c:\Users\krist\work\university\year2\semester2\data analysis\final-model\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\krist\work\university\year2\semester2\data analysis\final-model\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


ValueError: Provided exogenous values are not of the appropriate shape. Required (1185, 3), got (0, 3).

In [ ]:
plt.figure(figsize=(12, 6))
# Ensure datetime index and proper plotting
plt.plot(train_data.index, df["soil_moisture"], label="Actual Data")
plt.plot(test_data.index, prediction, label="Predicted")  # use the same x-axis as test

plt.legend()
plt.xlabel("Date")
plt.ylabel("Soil Moisture")
plt.title("Soil Moisture prediction")
plt.grid(True)
plt.tight_layout()
plt.show()